In [55]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


In [56]:
dir_path = "../process/"
file=  "external_test_data"#"train_data" #

logs_df = pd.read_csv(f"{dir_path}3_21{file}_heuristic_tagging.csv", encoding='utf-8')
print(set(logs_df["source"]))
print(logs_df.shape)
logs_df.head()

{'Apache', 'Android', 'Hadoop', 'Zookeeper', 'BGL'}
(320, 13)


,Content,Level,datetime,content_length,source,Clean_Content,Severidad,Origen,TipoEvento,Frecuencia,ContentLength,Hour,Anomaly
0,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:47:44.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1
1,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:52:15.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1
2,mod_jk child workerEnv in error state 7,Error,2005-12-04 04:52:52.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1
3,mod_jk child workerEnv in error state 7,Error,2005-12-04 04:53:16.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1
4,jk2_init() Found child 6755 in scoreboard slot 6,Informativo,2005-12-04 04:53:29.000,3,Apache,child scoreboard slot,Depuración,Conectividad,Monitoreo,Único,48,4.0,1


In [57]:
# Función para calcular la prioridad de un evento
def calcular_prioridad(row):
    prioridad = 0

    # Ponderación por Severidad
    if row['Severidad'] == 'Crítico':
        prioridad += 3  # Alta prioridad
    elif row['Severidad'] == 'Advertencia':
        prioridad += 2  # Prioridad media
    else:
        prioridad += 1  # Baja prioridad

    # Ponderación por Origen
    if row['Origen'] in ['Sistema operativo', 'Aplicación', ]:
        prioridad += 3  # Alta prioridad para estos orígenes
    elif row['Origen'] in ['Gestión de energía', 'Conectividad']:
        prioridad += 2  # Prioridad media
    else:
        prioridad += 1 

    # Ponderación por Tipo de Evento
    if row['TipoEvento'] in ['Errores', 'Parada de procesos']:
        prioridad += 3  # Alta prioridad
    elif row['TipoEvento'] in  ['Eventos']:
        prioridad += 2  # Prioridad media
    else:
        prioridad += 1 
        
    # Ponderación por Frecuencia
    if row['Frecuencia'] in ['Recurrente anómalo']:
        prioridad += 3  # Alta prioridad
    elif row['Frecuencia'] in ['Único']:
        prioridad += 2  # Prioridad media
    else:
        prioridad += 1  # Baja prioridad

    # Ponderación por Anomaly
    if row['Anomaly'] == -1:
        prioridad += 2  # Alta prioridad
    else:
        prioridad += 1  # Baja prioridad
        
    return prioridad

# Aplicar la función para calcular la prioridad
logs_df['Prioridad'] = logs_df.apply(calcular_prioridad, axis=1)


In [58]:
# Asignar etiquetas de prioridad en función del puntaje
def asignar_prioridad_etiqueta(row):
    if row['Prioridad'] > 10:
        return 'Error'
    elif row['Prioridad'] > 7:
        return 'Advertencia'
    else:
        return 'Informativo'


logs_df['PrioridadEtiqueta'] = logs_df.apply(asignar_prioridad_etiqueta, axis=1)
logs_df.head()

,Content,Level,datetime,content_length,source,Clean_Content,Severidad,Origen,TipoEvento,Frecuencia,ContentLength,Hour,Anomaly,Prioridad,PrioridadEtiqueta
0,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:47:44.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1,11,Error
1,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:52:15.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1,11,Error
2,mod_jk child workerEnv in error state 7,Error,2005-12-04 04:52:52.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1,11,Error
3,mod_jk child workerEnv in error state 7,Error,2005-12-04 04:53:16.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1,11,Error
4,jk2_init() Found child 6755 in scoreboard slot 6,Informativo,2005-12-04 04:53:29.000,3,Apache,child scoreboard slot,Depuración,Conectividad,Monitoreo,Único,48,4.0,1,7,Informativo


In [59]:
logs_df.to_csv(f"{dir_path}3_22{file}_heuristic_prediction.csv", index=False, encoding='utf-8')

In [60]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [61]:
print({file})
# Asumiendo que logs_df ya está definido y contiene las columnas 'Level' y 'PrioridadEtiqueta'
# Asegúrate de que ambos sean de tipo categórico (opcional pero recomendado)
logs_df['Level'] = logs_df['Level'].astype('category')
logs_df['PrioridadEtiqueta'] = logs_df['PrioridadEtiqueta'].astype('category')

# Extraer columnas
y_true = logs_df['Level']
y_pred = logs_df['PrioridadEtiqueta']

# Calcular accuracy
accuracy = accuracy_score(y_true, y_pred)

# Obtener etiquetas únicas combinadas para garantizar alineación
all_labels = sorted(set(y_true.unique()) | set(y_pred.unique()))

# Matriz de confusión
conf_matrix = confusion_matrix(y_true, y_pred, labels=all_labels)

# Reporte de clasificación
report = classification_report(y_true, y_pred, labels=all_labels, output_dict=True)

# Convertir a DataFrame
report_df = pd.DataFrame(report).transpose()

# Mostrar resultados
print(f"Accuracy: {accuracy:.2f}\n")
print("Classification Report:")
print(report_df)
print("\nConfusion Matrix:")
print(pd.DataFrame(conf_matrix, index=all_labels, columns=all_labels))

{'external_test_data'}
Accuracy: 0.90

Classification Report:
              precision    recall  f1-score     support
Advertencia    0.602941  0.931818  0.732143   44.000000
Error          0.992424  0.949275  0.970370  138.000000
Informativo    0.975000  0.847826  0.906977  138.000000
accuracy       0.903125  0.903125  0.903125    0.903125
macro avg      0.856788  0.909640  0.869830  320.000000
weighted avg   0.931356  0.903125  0.910276  320.000000

Confusion Matrix:
             Advertencia  Error  Informativo
Advertencia           41      0            3
Error                  7    131            0
Informativo           20      1          117
